In [47]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

pd.plotting.register_matplotlib_converters()
%matplotlib inline

import duckdb


%load_ext autoreload
%autoreload 2
from help_tool import help_tool

#pd.set_option('future.no_silent_downcasting', True)

# Setting graph parameters
sns.set_theme(style='white')

pd.set_option('display.max_colwidth', None)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [62]:
train = help_tool.csv_download(
    r'Archive\train.csv')

help_tool.first_look(train)

Dataset has 159571 observations and 8 features
Columns with all empty values []
Dataset has 0 duplicates


,dtypes,"Null values, Count","Null values, %"
id,object,0,0.0
comment_text,object,0,0.0
toxic,int64,0,0.0
severe_toxic,int64,0,0.0
obscene,int64,0,0.0
threat,int64,0,0.0
insult,int64,0,0.0
identity_hate,int64,0,0.0


In [49]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27",0,0,0,0,0,0
1,000103f0d9cfb60f,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of accidents"""" -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It's listed in the relevant form eg Wikipedia:Good_article_nominations#Transport """,0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember what page that's on?",0,0,0,0,0,0


In [63]:
train[train['id'] == '83e37ee1eb4be93e']

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
49323,83e37ee1eb4be93e,"""== Blocked as a sockpuppet ==\n""",1,0,0,0,0,0


In [50]:
train[train['id'].duplicated()]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate


In [51]:
train[train['comment_text'].duplicated()]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate


In [6]:
train.drop(columns=['id', 'comment_text']).describe()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [7]:
train.drop(columns=['id', 'comment_text']).drop_duplicates().head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0,0,0,0,0,0
6,1,1,1,0,1,0
12,1,0,0,0,0,0
42,1,0,1,0,1,1
43,1,0,1,0,1,0


In [8]:
train.drop(columns=['id', 'comment_text']).sum()

toxic            15294
severe_toxic      1595
obscene           8449
threat             478
insult            7877
identity_hate     1405
dtype: int64

In [9]:
train[train['toxic'] == 1]['comment_text'].head()

6                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
12                                                                                                                                                                                                                                                                                                                                         

In [10]:
#def clean_text(df):
#train['comment_text'] = train['comment_text'].replace({'\\n': '', '  ': ' '}, regex=True)

In [11]:
train[train['comment_text'].str.contains(r'Image:.*\.jpg', regex=True)]['comment_text']

10                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   "\nFair use rationale for Image:Wonju.jpg\n\nThanks for uploading Image:Wonju.jpg. I notice the image page specifies that the image is being used under fair use but there is no explanation or rationale as to why its use in Wikipedia articles constitutes fair use. In addition to the boilerplate fair use template, you must also write out on the image description page a specific explanation or rationale

In [18]:
train[~train['comment_text'].str.contains(r'==.*==\n', regex=True)]['comment_text']

0                                                                                                                                                                                                                                                                                                                                                                                  Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
1                                                                                                                                                                                                                                                                                                                                                                          

Title is removed. not

In [24]:
train[train['comment_text'].str.contains(r'==.*==\n', regex=True)]['comment_text'].head(1)

29    "== A barnstar for you! ==\n\n  The Real Life Barnstar lets us be the stars\n   "
Name: comment_text, dtype: object

In [53]:
#train['comment_text'] = train['comment_text'].str.replace(r'==.*==\n', '', regex=True)

In [91]:
train['comment_text'] = train['comment_text'].replace({'\n': '.', '==': ''})

Non ascii

In [80]:
train['comment_text'].str.encode("ascii", errors="ignore").iloc[0]

b"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27"

In [77]:
import hashlib

def hash_text(text):
    return hashlib.sha256(text.encode()).hexdigest()

train['hash_value'] = train['comment_text'].apply(hash_text)

In [76]:
train[train['comment_text'].duplicated()]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate


In [78]:
train[train['hash_value'].duplicated()]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,hash_value


In [38]:
train[train['comment_text'].str.contains(r'b".*\n', regex=True)]['comment_text']

1908                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [105]:
train[train['comment_text'].str.contains(r'0-9')]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,hash_value,contains_non_english
901,027c1f1ec6bba226,"Lol, you know what i meant. Whichever president came before Hafez (i just new he was president before assad) has considerable foreground of support, in which his supports would have freaked out because of the coup. They would have claimed instability would arise, as assad's supporters are trying to use in their defense. \n\nWhat i am trying to say is that although Bashar assad had good economic and international policies for Syria, you should not fear a non-baathist potentially being elected. Syria's economic and international policies won't change (iran would still be a major factor in Syrian life considering the buisness deals stakes and cultural exchanges) , the only this that would change are social freedoms, and for the better. Also every country had a revolution and they are still alive today. The Muslim brotherhood in Syria is weak, even weaker than the islamist prescence in Libya. The rebels in Libya had 80-90% support, as we see now, and yet could hardly take control without NATO. What chance does the Syrian Muslim brotherhood have? None. If anything, nonviolent protests weaken the resolve of the Muslim brotherhood, making them less of an influence in the everyday life of Syrians. As you know I am pro Syria and Armenia, just not pro Assad. Unlike Armenia where people like Sargsyan and Kopcharyan were democratically elected, Assad was not (no one ran against him because they were not allowed).",0,0,0,0,0,0,23b204912ae4f4e3eea5e5c381155b5106ad370dc7bb1f4f9358aca4b96236d3,True
4701,0c768d0a188e93ed,"""\n\n Controversial Remarks Alleged Calling Facebook Users """"Dumb F**ks"""" \n\nQuote: To avoid being blocked, instead of reverting please consider using the article's talk page to work toward making a version that represents consensus among editors. \n\n!!!!THAT IS WHAT I AM DOING!!!!\n\nREFRAIN FROM DELETING THE TALK PAGE CONTRIBUTION WITHOUT PROVIDING A VALID REASON OR BETTER TIPS FOR IMPROVEMENT. AFTER ALL THIS IS ONLY THE TALK PAGE\n\nsources for the below mentioned contribution:\n\n http://www.forbes.com/sites/andygreenberg/2012/02/01/is-zuckerbergs-the-hacker-way-letter-facebooks-dont-be-evil-and-will-it-live-up-to-it/\n http://gawker.com/5636765/facebook-ceo-admits-to-calling-users-dumb-fucks\n http://au.businessinsider.com/well-these-new-zuckerberg-ims-wont-help-facebooks-privacy-problems-2010-5\n http://www.theregister.co.uk/2010/05/14/facebook_trust_dumb/\n http://www.theweek.co.uk/technology/14625/are-users-%E2%80%98dumb-fucks%E2%80%99-trusting-data-facebook\n http://tdh.me/zuckerberg-called-early-facebook-users-dumb-fucks-so-what/\n http://anphicle.com/en/they-trust-me-dumb-fucks-facebook-ceo-mark-zuckerberg/\n http://www.tomsguide.com/us/Facebook-Mark-Zuckerberg-Social-Networking-privacy-security,news-6794.html\n http://au.businessinsider.com/embarrassing-and-damaging-zuckerberg-ims-confirmed-by-zuckerberg-the-new-yorker-2010-9\n\nOther sections on Wikipedia where Controversial Remarks is valid: \n\n Bryant Gumbel (section Controversial remarks)\n Mel Gibson (section Controversial remarks)\n Sherrod Brown (section Controversial remarks)\n Kent Hovind (section Controversial remarks)\n Venkaiah Naidu (section Controversial Remarks)\n Greg Gutfeld (section Controversial remarks)\n Lawrence Cannon (section Controversial remarks)\n Daniel Bernard (diplomat) (section Controversial remark)\n James R. Flynn (section Controversial remarks)\n\nand many many more\n\nstart.\n\nWhile at Harvard College 19-year-old Facebook CEO Mark Zuckerberg allegedly used login data from his social network to hack into fellow students email accounts; that evidence also included instant message transcripts where he calls facebook users """"dumb fucks"""". The alleged following exchange is between a 19-year-old Mark Zuckerberg and a friend shortly after Mark launched The Facebook in his dorm room:\n \nZuck: Yeah so

In [101]:
non_english_pattern = r'[^a-zA-Z\s\.,!?:-]'

# Apply the regex pattern to detect non-English letters
train['contains_non_english'] = train['comment_text'].str.contains(non_english_pattern)

# Display rows with non-English letters
non_english_rows = train[train['contains_non_english']]

In [102]:
non_english_rows['comment_text']

0                                                                                                                                                                                                                                                                                                                                                                                                             Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27
1                                                                                                                                                                                                                                                                                                                                               